<a href="https://colab.research.google.com/github/futugyou/pyproject/blob/master/google_colab/generation_embedding_model_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required dependencies
%pip install datasets
%pip install sentence_transformers
%pip install transformers
%pip install torch
%pip install tqdm
%pip install scikit-learn
%pip install mteb
%pip install pandas
%pip install nltk

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
# The code of generate-modle is almost identical to finetune-model
import pandas as pd
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, InputExample
from datasets import load_dataset, Dataset
from sentence_transformers import InputExample
from sentence_transformers.dataset import DenoisingAutoEncoderDataset

mnli = load_dataset("glue", "mnli", split="train").select(range(25_000))
flat_sentences = mnli["premise"], mnli["hypothesis"]
damaged_data = DenoisingAutoEncoderDataset(list(set(flat_sentences)))

train_dataset = {"damaged_sentences": [], "original_sentences": []}
for data in tqdm(damaged_data):
    train_dataset["damaged_sentences"].append(data[0])
    train_dataset["original_sentences"].append(data[1])
    
train_dataset = Dataset.from_dict(train_dataset)

In [ ]:
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

val_sts = load_dataset("glue", "stsb", split="validation")
evaluator = EmbeddingSimilarityEvaluator(
    sentence1=val_sts["sentence1"],
    sentence2=val_sts["sentence2"],
    scores=[score/5 for score in val_sts["label"]],
    main_similarity="cosine",
)

In [ ]:
from sentence_transformers import SentenceTransformer, models

word_embeddings = models.Transformer('bert-base-uncased')
pooling_model = models.Pooling(word_embeddings.get_word_embedding_dimension(), "cls")
mebedding_model = SentenceTransformer(modules=[word_embeddings, pooling_model])

In [ ]:
from sentence_transformers import  losses
from sentence_transformers.trainer import SentenceTransformerTrainer
from sentence_transformers.training_args import SentenceTransformerTrainingArguments

# loss function
train_loss = losses.DenisingAutoEncoderLoss(model=mebedding_model)
train_loss.decoder = train_loss.decoder.to("cuda")

# training arguments
args = SentenceTransformerTrainingArguments(
    output_dir="tsdae_mebedding_model",
    num_train_epochs=1,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=100,
    fp16=True,
    eval_steps=100,
    logging_steps=100,
)

In [ ]:

# train model
trainer = SentenceTransformerTrainer(
    model=embedding_model,
    args=args,
    train_dataset=train_dataset,
    loss=train_loss,
    evaluator=evaluator,
)
trainer.train()

In [ ]:
# evaluator model
evaluator(embedding_model)